In [0]:
import pandas as pd

In [0]:
from sklearn.datasets import load_boston

boston = load_boston()
X = pd.DataFrame(boston.data)
X.columns = [_.lower() for _ in boston.feature_names]
y = pd.DataFrame(boston.target)
y.columns=['target']
df = pd.concat([X,y], axis=1)
df.head(5)

Out[87]:

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [0]:
df.columns

Out[88]: Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
 'ptratio', 'b', 'lstat', 'target'],
 dtype='object')

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 crim 506 non-null float64
 1 zn 506 non-null float64
 2 indus 506 non-null float64
 3 chas 506 non-null float64
 4 nox 506 non-null float64
 5 rm 506 non-null float64
 6 age 506 non-null float64
 7 dis 506 non-null float64
 8 rad 506 non-null float64
 9 tax 506 non-null float64
 10 ptratio 506 non-null float64
 11 b 506 non-null float64
 12 lstat 506 non-null float64
 13 target 506 non-null float64
dtypes: float64(14)
memory usage: 55.5 KB

In [0]:
spark_df = spark.createDataFrame(df)
display(spark_df.head(5))

crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98,24.0
0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14,21.6
0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.9,5.33,36.2


#Pipeline

In [0]:
trainDF, testDF = spark_df.randomSplit([0.8, 0.2], seed=42)
print(trainDF.cache().count())
print(testDF.count())

396
110

In [0]:
display(trainDF.head(5))

crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98,24.0
0.01311,90.0,1.22,0.0,0.403,7.249,21.9,8.6966,5.0,226.0,17.9,395.93,4.81,35.4
0.01432,100.0,1.32,0.0,0.411,6.816,40.5,8.3248,5.0,256.0,15.1,392.9,3.95,31.6
0.02055,85.0,0.74,0.0,0.41,6.383,35.7,9.1876,2.0,313.0,17.3,396.9,5.77,24.7
0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7


In [0]:
from pyspark.ml.feature import VectorAssembler

assemblerInputs = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax','ptratio', 'b', 'lstat']
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="target", maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [0]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssembler, lr])
pipelineModel = pipeline.fit(trainDF)

predDF = pipelineModel.transform(testDF)

In [0]:
display(predDF.select("features", "target", "prediction").show(5))

+--------------------+------+------------------+
 features|target| prediction|
+--------------------+------+------------------+
[0.0136,75.0,4.0,...| 18.9| 17.63639242965074|
[0.02731,0.0,7.07...| 21.6| 25.34987201081645|
[0.02985,0.0,2.18...| 28.7|25.873740546634522|
[0.0536,21.0,5.64...| 25.0| 27.64149982939145|
[0.09378,12.5,7.8...| 21.7|21.325557546852238|
+--------------------+------+------------------+
only showing top 5 rows

#No Pipeline

In [0]:
from pyspark.ml.feature import VectorAssembler

assemblerInputs = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax','ptratio', 'b', 'lstat']
vectorAssembler = VectorAssembler(inputCols = assemblerInputs, outputCol = 'features')
spark_df_v = vectorAssembler.transform(spark_df)
display(spark_df_v.head(5))

crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target,features
0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98,24.0,"Map(vectorType -> dense, length -> 13, values -> List(0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98))"
0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14,21.6,"Map(vectorType -> dense, length -> 13, values -> List(0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14))"
0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,"Map(vectorType -> dense, length -> 13, values -> List(0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, 4.9671, 2.0, 242.0, 17.8, 392.83, 4.03))"
0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,"Map(vectorType -> dense, length -> 13, values -> List(0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, 6.0622, 3.0, 222.0, 18.7, 394.63, 2.94))"
0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.9,5.33,36.2,"Map(vectorType -> dense, length -> 13, values -> List(0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, 6.0622, 3.0, 222.0, 18.7, 396.9, 5.33))"


In [0]:
trainDF, testDF = spark_df_v.randomSplit([0.8, 0.2], seed=42)
print(trainDF.cache().count())
print(testDF.count())

396
110

In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='target', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(trainDF)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.035253982399516944,0.011349927542578547,0.0,3.0279774412827503,-9.216539898630172,4.266495950497568,0.0,-0.6579579709156765,0.0,0.0,-0.7966807853226541,0.00857557992467222,-0.4696082115584449]
Intercept: 20.615853909008734

In [0]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.986729
r2: 0.710273

In [0]:
lr_predictions = lr_model.transform(testDF)
display(lr_predictions.select("target","features","prediction").show(5))

+------+--------------------+------------------+
target| features| prediction|
+------+--------------------+------------------+
 18.9|[0.0136,75.0,4.0,...| 17.63639242965074|
 21.6|[0.02731,0.0,7.07...| 25.34987201081645|
 28.7|[0.02985,0.0,2.18...|25.873740546634522|
 25.0|[0.0536,21.0,5.64...| 27.64149982939145|
 21.7|[0.09378,12.5,7.8...|21.325557546852238|
+------+--------------------+------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="target",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

R Squared (R2) on test data = 0.739241

In [0]:
test_result = lr_model.evaluate(testDF)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 4.52443